### Carregando o modelo

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import json
import string
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [7]:
from keras.models import load_model

In [8]:
def loadModel():
    name = input('escreva o nome do modelo a ser carregado:')
    savedModel = load_model(name+'.h5')
    with open('wordIndex.json') as f:
        dicto = json.load(f)
    savedModel.summary()
    return savedModel, dicto

In [9]:
tokenizer = Tokenizer()
model, tokenizer.word_index = loadModel()

escreva o nome do modelo a ser carregado:0526_rnn_truncado100_acc81
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          9235900   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 9,353,277
Trainable params: 117,377
Non-trainable params: 9,235,900
_________________________________________________________________


### Notícias analisadas

In [57]:
sentence_t = '''A defensora pública aposentada Cláudia Alvarim Barrozo, 59, gravada agredindo verbalmente entregadores negros, no último sábado (30), em vídeo em que chama um deles de 'macaco', em um condomínio de luxo de Niterói (RJ), tem cinco passagens anteriores pela polícia por injúria racial. Além disso, a reportagem do UOL apurou que as investigações tiveram início em 2014 e que há ainda uma sexta anotação policial antes do caso deste final de semana, mas por lesão corporal e constrangimento.

Claudia Barrozo foi intimada a prestar esclarecimentos na tarde desta quinta-feira (5) na 81ª DP (Itaipu), porém alegou que já tinha "compromissos" e pediu para que a oitiva fosse adiada para a semana que vem.

Hoje pela manhã, Eduardo Pessanha Marques e Jonathas de Souza Mendonça estiveram na delegacia para serem ouvidos novamente. "A sensação que eu tive era de que eu estava em um filme, em uma novela e que aquilo não poderia estar acontecendo", disse Eduardo, que gravou o vídeo flagrando o crime.

De acordo com o delegado Carlos César Santos, atualmente "as pessoas denunciam, não se calam, por isso a incidência [do crime de injúria racial] é maior hoje em dia". Os agentes irão analisar imagens de câmeras de segurança.'''

In [86]:
sentence_t = '''O ex-juiz e ex-ministro Sergio Moro (União Brasil) defendeu nesta 5ª feira (26.mai.2022) a PRF (Polícia Rodoviária Federal) e disse que a atuação dos 2 agentes da corporação no caso do homem asfixiado em Sergipe é uma “exceção”. Genivaldo de Jesus Santos morreu depois de ser colocado, por policiais rodoviários federais, dentro do porta-malas de uma viatura com fumaça dentro.

“Lamentável ação em Sergipe de 2 policiais da PRF. Mas que não se tome exceção como regra. Conheci de perto a PRF quando ministro. São profissionais valorosos e a violência policial é rara. Que tudo seja apurado e os culpados, punidos. Meus sentimentos à família do senhor Genivaldo”, escreveu no Twitter. 

Moro deixou o cargo de ministro da Justiça do governo Bolsonaro em abril de 2020 depois que o presidente demitiu o então diretor-geral da PF Maurício Valeixo, que havia sido escolhido pelo ex-juiz da Lava Jato. As declarações de Moro quando pediu demissão são a base de uma investigação sobre a suposta interferência política de Bolsonaro na PF.'''

In [94]:
sentence_t = '''Os Estados Unidos continuam em choque após a morte de 21 pessoas, entre elas 19 crianças, em um ataque em uma escola na cidade de Uvalde, no Texas. O presidente Joe Biden anunciou na quarta-feira (25) que iria ao estado nos próximos dias, onde políticos pró e contra armas se opõem.

Eram 11h30 em Uvalde, no estado americano do Texas, quando Salvador Ramos começou a atirar em alunos e professores da escola de ensino fundamental Robb, matando 21 pessoas, entre elas 19 crianças, na terça-feira (24).

Celeste estava no carro quando ouviu no rádio que um incidente estava acontecendo na escola de sua filha de 9 anos. A mãe de família, de 30 anos, correu para o local. Quando chegou, os policiais a impediram de entrar na escola e, na mesma hora, o atirador saiu do prédio e atirou na direção dos agentes.
"Ele saiu e começou a atirar em todo mundo. Eu fiquei com muito medo", conta. Poucos minutos depois, tiros voltaram a ser ouvidos no interior do centro de ensino. A filha de Celeste ainda estava no prédio.

"Ela ficou escondida no banheiro o tempo todo. Ela tentou sair e ele atirou bem na frente dela. Então ela parou e se escondeu em outro lugar. Ela então viu o atirador correr e atirar e se escondeu durante 15 minutos e depois começou a correr para a primeira porta aberta", conta Celeste, que conseguiu encontrar a filha viva.

"Nessa hora eu pulei, a agarrei e a coloquei dentro do meu carro. Então eu esperei que os filhos de duas amigas minhas saíssem, mas eles não saíram", diz a mãe emocionada.

Celeste acredita que o que aconteceu com ela e a filha foi um "milagre". Ela conhecia todas as crianças e não pode evitar se sentir culpada por sua filha ter sobrevivido. Por isso, ela decidiu depositar, diante do portão da escola, 19 bichos de pelúcia, um para cada vítima.

"Eu trago um bicho de pelúcia para cada um para que eles possam dormir com eles. Eles são tão pequenos. São bebês. Esta é para Eli", diz mostrando um dos brinquedos. "Era uma das filhas de uma amiga minha. Ela fazia sempre vídeos super fofos. A mãe dela é uma boa amiga", contou ao enviado especial da RFI em Uvalde, David Thomson.

"Todas estas crianças eram conhecidas. Elas eram todas tão boas. Crianças inocentes. Bebês. Alguns vinham às festas da minha família. Outras eram filhos de amigos", diz Celeste chocada com o que foi o pior ataque a um centro de ensino nos Estados Unidos, em 10 anos.

"Todo mundo se conhece aqui, é uma pequena comunidade. É simplesmente muito triste. Somos todos próximos. Minha vizinha perdeu a sobrinha e o sobrinho. Meu coração está partido porque minha filha também estava na escola, mas ela voltou para casa. As crianças de meus amigos não voltaram", lamenta.

No dia seguinte da matança, centenas de habitantes de Uvalde se reuniram para homenagear as vítimas. Orações, músicas e sofrimento marcaram a manifestação. Entre os presentes, um agente de polícia em uniforme, pai de uma vítima, arrasado, em lágrimas nos braços do senador Ted Cruz, que estava presente, assim como Greg Abott, governador do Texas.

Uvalde é uma pequena comunidade rural de 16 mil habitantes, três quartos hispânicos. "Estamos simplesmente chocados. Não conseguimos acreditar que uma coisa assim tenha acontecido em uma cidade tão pequena", disse o pai de um aluno.

"Eu estou triste, simplesmente triste. Faltam palavras. Grotesco, desprezível. Eram crianças inocentes. Elas tinham a vida diante delas, um futuro. Tudo isso foi roubado", disse outro pai de um sobrevivente.'''

In [65]:
sentence_f = '''Bolsonaro põe fim ao seu casamento e isolada ex-mulher O casamento de Michelle e Jair Bolsonaro chegou ao fim. O casal passa por uma crise complexa desde novembro de 2018, depois que Jair foi eleito Presidente do Brasil.
 
O site de cobertura e fatos de celebridades e famosos, [site] foi o primeiro site a revelar a separação do casal no dia 13 de outubro de 2020. Na terça-feira o mesmo site revelou que a ex-primeira dama e esposa de Bolsonaro – Michelle Bolsonaro tem sofrido com o fim do casamento e se sentido isolada.

O Custo da vila Olímpica do Japão: 29 milhões de dólares. O custo da vila Olímpica na olimpíada do Brasil: 900 milhões de dólares. e pra variar quem governava o Brasil na época: PT e Lula, quem ganhou muito dinheiro: políticos envolvidos na corrupção, e a globolixo

NO RIO GRANDE DO SUL PRODUTORES DE UVA TOMAM MEDIDAS PARA EVITAR A GEADA NO RIO GRANDE DO SUL PRODUTORES DE UVA TOMAM MEDIDAS PARA EVITAR A GEADA Conforme informações do diário da informação em Bento Gonçalves para não congelar as uvas os produtores estão utilizando o fogo em tonéis, a técnica já é usada há alguns anos no Brasil e em outros países. Além da proteção é mais um espetáculo na Serra Gaúcha.

Lula admite DERROTA em discurso de 1º de Maio. O discurso de Lula que era para ser às 12:00hs foi mudado para às 13:00 e só começou às 15:30. Tudo porque não tinha gente o suficiente. No discurso Lula diz que ainda não é candidato. Ele disse: “só dia 7 eu vou ser pré-candidato”. Será que já arregou?

Bolsonaro põe fim ao seu casamento e isolada ex-mulher O casamento de Michelle e Jair Bolsonaro chegou ao fim. O casal passa por uma crise complexa desde novembro de 2018, depois que Jair foi eleito Presidente do Brasil.
 
O site de cobertura e fatos de celebridades e famosos, [site] foi o primeiro site a revelar a separação do casal no dia 13 de outubro de 2020. Na terça-feira o mesmo site revelou que a ex-primeira dama e esposa de Bolsonaro – Michelle Bolsonaro tem sofrido com o fim do casamento e se sentido isolada.

O Custo da vila Olímpica do Japão: 29 milhões de dólares. O custo da vila Olímpica na olimpíada do Brasil: 900 milhões de dólares. e pra variar quem governava o Brasil na época: PT e Lula, quem ganhou muito dinheiro: políticos envolvidos na corrupção, e a globolixo

NO RIO GRANDE DO SUL PRODUTORES DE UVA TOMAM MEDIDAS PARA EVITAR A GEADA NO RIO GRANDE DO SUL PRODUTORES DE UVA TOMAM MEDIDAS PARA EVITAR A GEADA Conforme informações do diário da informação em Bento Gonçalves para não congelar as uvas os produtores estão utilizando o fogo em tonéis, a técnica já é usada há alguns anos no Brasil e em outros países. Além da proteção é mais um espetáculo na Serra Gaúcha.

Lula admite DERROTA em discurso de 1º de Maio. O discurso de Lula que era para ser às 12:00hs foi mudado para às 13:00 e só começou às 15:30. Tudo porque não tinha gente o suficiente. No discurso Lula diz que ainda não é candidato. Ele disse: “só dia 7 eu vou ser pré-candidato”. Será que já arregou?'''

In [72]:
sentence_f = '''bolsonaro',
 'explica',
 'em',
 'vídeo',
 'porque',
 'o',
 'filho',
 'poderia',
 'ser',
 'preso',
 'em',
 'vídeo',
 'o',
 'deputado',
 'federal',
 'jair',
 'bolsonaro',
 'explica',
 'as',
 'razões',
 'porque',
 'disse',
 'que',
 'seu',
 'filho',
 'eduardo',
 'bolsonaro',
 'poderia',
 'ser',
 'preso',
 'no',
 'complexo',
 'penitenciário',
 'da',
 'papuda',
 'veja',
 'o',
 'vídeo',
 'e',
 'tire',
 'suas',
 'conclusões',
 'o',
 'deputado',
 'federal',
 'jair',
 'bolsonaro',
 'pscrj',
 'disse',
 'que',
 'vai',
 'processar',
 'por',
 'invasão',
 'de',
 'privacidade',
 'o',
 'fotógrafo',
 'que',
 'publicou',
 'no',
 'facebook',
 'imagens',
 'que',
 'mostram',
 'o',
 'parlamentar',
 'travando',
 'um',
 'diálogo',
 'duro',
 'via',
 'whatsapp',
 'com',
 'o',
 'também',
 'deputado',
 'eduardo',
 'bolsonaro',
 'pscsp',
 'seu',
 'filho',
 'de',
 'anos',
 'o',
 'parlamentar',
 'afirmou',
 'que',
 'não',
 'vai',
 'discutir',
 'o',
 'teor',
 'ou',
 'a',
 'autencidade',
 'das',
 'mensagensde',
 'acordo',
 'com',
 'jair',
 'bolsonaro',
 'a',
 'atitude',
 'de',
 'lula',
 'marques',
 'consiste',
 'em',
 'invasão',
 'de',
 'privacidade\x96',
 'não',
 'vou',
 'discutir',
 'o',
 'teor',
 'das',
 'mensagens',
 'se',
 'é',
 'verdade',
 'ou',
 'não',
 'vou',
 'processar',
 'o',
 'fotógrafo',
 'que',
 'já',
 'sei',
 'quem',
 'é',
 'por',
 'invasão',
 'de',
 'privacidade',
 '\x96',
 'disse',
 'o',
 'deputado',
 'ao',
 'extraa',
 'veja',
 'mostrou',
 'imagens',
 'do',
 'telefone',
 'celular',
 'de',
 'jair',
 'bolsonaronaquele',
 'momento',
 'ele',
 'estava',
 'xingando',
 'seu',
 'filho',
 'eduardo',
 'bolsonaro',
 'pelo',
 'whatsappmotivo',
 'eduardo',
 'bolsonaro',
 'não',
 'participara',
 'da',
 'votação',
 'para',
 'a',
 'presidência',
 'da',
 'câmara',
 'dos',
 'deputados',
 'embora',
 'seu',
 'pai',
 'fosse',
 'um',
 'dos',
 'candidatos',
 'ao',
 'cargojair',
 'bolsonaro',
 'escreveu\x93papel',
 'de',
 'filho',
 'da',
 'puta',
 'que',
 'você',
 'está',
 'fazendo',
 'comigo',
 'irresponsável\x94em',
 'seguida\x93mais',
 'ainda',
 'compre',
 'merdas',
 'por',
 'aí',
 'não',
 'vou',
 'te',
 'visitar',
 'na',
 'papuda\x94e',
 'também\x93se',
 'a',
 'imprensa',
 'te',
 'descobrir',
 'aí',
 'e',
 'o',
 'que',
 'está',
 'fazendo',
 'vão',
 'comer',
 'seu',
 'figado',
 'e',
 'o',
 'meu',
 'retorne',
 'imediatamente\x94jair',
 'bolsonaro',
 'anunciou',
 'que',
 'vai',
 'processar',
 'a',
 'veja',
 'pelas',
 'imagens',
 'publicadas'''

### Preprocessing

In [12]:
# Carregando o pacote de língua portuguesa para o processador Spacy
nlp = spacy.load('pt_core_news_sm')

In [13]:
# Defininido funções de preprocessamento

def removePunct(text):
    '''
    Removes any punctuation included in string.punctuation.
    '''
    translator = text.maketrans({key:'' for key in string.punctuation+'“”'}) # Translates any punctuation into ''
    return text.translate(translator)

def removeNumbers(text):
    '''
    Removes any number character in text.
    '''
    return re.sub('[0-9]', '' , text) # Translates any number into ''

def removeStopWords(string):
    '''
    Removes any portuguese stopwords, using Spacy's standard package.
    '''
    doc = nlp(string)
    return ' '.join([token.text for token in doc if token.is_stop is False])

def lemmatize(string):
    '''
    Lemmatizes text word-by-word. Notice that lemmatizing is not as harsh as stemming, which makes the final text easier to read and understand in common language.
    '''
    doc = nlp(string)
    return ' '.join([token.lemma_ for token in doc])

def prep(string, useStopWords = False, lemma = False):
    '''
    Executes previously defined preprocessing in text.
    '''

    result = removeNumbers(removePunct(string)).lower()
    
    if useStopWords and lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc if token.is_stop is False])
    elif useStopWords:
        doc = nlp(result)
        result = ' '.join([token.text for token in doc if token.is_stop is False])
    elif lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc])

    result = result.replace('\n',"")
    
    return result

In [95]:
# Realizando preprocessamento de textos em notícias de teste presentes no DataFrame.

sentence_t = prep(sentence_t)

In [80]:
# Realizando preprocessamento de textos em notícias de teste presentes no DataFrame.

sentence_f = prep(sentence_f)

In [96]:
sentence_t

'os estados unidos continuam em choque após a morte de  pessoas entre elas  crianças em um ataque em uma escola na cidade de uvalde no texas o presidente joe biden anunciou na quartafeira  que iria ao estado nos próximos dias onde políticos pró e contra armas se opõemeram h em uvalde no estado americano do texas quando salvador ramos começou a atirar em alunos e professores da escola de ensino fundamental robb matando  pessoas entre elas  crianças na terçafeira celeste estava no carro quando ouviu no rádio que um incidente estava acontecendo na escola de sua filha de  anos a mãe de família de  anos correu para o local quando chegou os policiais a impediram de entrar na escola e na mesma hora o atirador saiu do prédio e atirou na direção dos agentesele saiu e começou a atirar em todo mundo eu fiquei com muito medo conta poucos minutos depois tiros voltaram a ser ouvidos no interior do centro de ensino a filha de celeste ainda estava no prédioela ficou escondida no banheiro o tempo todo 

In [81]:
sentence_f

'quatro expresidentes presos até é o que a lava jato está preparando a força tarefa da operação lava jato está com farto material acusatório para levar para a cadeia simultaneamente expresidentes do brasil lula dilma temer e collor podem terminar sua vida política atrás das gradesa condenação de lula acendeu o alerta vermelho em políticos que se achavam intocáveis a operação lava jato ao condenar o maior líder político da esquerda no brasil demonstrou que nenhuma pessoa está acima da leia condenação a nove anos e seis meses de prisão pelo juiz federal sergio moro nesta quartafeira não mandará o petista automaticamente para a cadeia isso porque moro é um juiz de primeira instância e assim sua decisão não basta para que a pena seja cumprida imediatamentetoda decisão que o \x93juiz da lava jato\x94 toma em primeira instância é depois revisada em segunda instância pelos desembargadores do tribunal regional da a região trf joão pedro gebran neto relator da operação leandro paulsen e victor 

### Classificação Trues

In [105]:
lista_t = []
lista_t.append(sentence_t)
lista_t

['os estados unidos continuam em choque após a morte de  pessoas entre elas  crianças em um ataque em uma escola na cidade de uvalde no texas o presidente joe biden anunciou na quartafeira  que iria ao estado nos próximos dias onde políticos pró e contra armas se opõemeram h em uvalde no estado americano do texas quando salvador ramos começou a atirar em alunos e professores da escola de ensino fundamental robb matando  pessoas entre elas  crianças na terçafeira celeste estava no carro quando ouviu no rádio que um incidente estava acontecendo na escola de sua filha de  anos a mãe de família de  anos correu para o local quando chegou os policiais a impediram de entrar na escola e na mesma hora o atirador saiu do prédio e atirou na direção dos agentesele saiu e começou a atirar em todo mundo eu fiquei com muito medo conta poucos minutos depois tiros voltaram a ser ouvidos no interior do centro de ensino a filha de celeste ainda estava no prédioela ficou escondida no banheiro o tempo todo

In [106]:
maxlen = 100
sequences_t = tokenizer.texts_to_sequences(lista_t)
padded_t = pad_sequences(sequences_t, maxlen=maxlen)

In [107]:
model.predict(padded_t)

array([[0.9815802]], dtype=float32)

In [100]:
(model.predict(padded_t) >=0.5).astype(int)

array([[1]])

### Classificação Fakes

In [82]:
lista_f = []
lista_f.append(sentence_f)
lista_f

['quatro expresidentes presos até é o que a lava jato está preparando a força tarefa da operação lava jato está com farto material acusatório para levar para a cadeia simultaneamente expresidentes do brasil lula dilma temer e collor podem terminar sua vida política atrás das gradesa condenação de lula acendeu o alerta vermelho em políticos que se achavam intocáveis a operação lava jato ao condenar o maior líder político da esquerda no brasil demonstrou que nenhuma pessoa está acima da leia condenação a nove anos e seis meses de prisão pelo juiz federal sergio moro nesta quartafeira não mandará o petista automaticamente para a cadeia isso porque moro é um juiz de primeira instância e assim sua decisão não basta para que a pena seja cumprida imediatamentetoda decisão que o \x93juiz da lava jato\x94 toma em primeira instância é depois revisada em segunda instância pelos desembargadores do tribunal regional da a região trf joão pedro gebran neto relator da operação leandro paulsen e victor

In [103]:
maxlen = 100
sequences_f = tokenizer.texts_to_sequences(lista_f)
padded_f = pad_sequences(sequences_f, maxlen=maxlen)

In [104]:
model.predict(padded_f)

array([[0.9640597]], dtype=float32)

In [85]:
(model.predict(padded_f) >=0.5).astype(int)

array([[1]])